In [88]:
from typing import Iterable, List, Dict
from collections import defaultdict
from operator import itemgetter

In [56]:
cargos = [int(c) for c in "23 3 51 20 51 42 52 12 4 7 59 58 25 94 18".split()]
cargos

[23, 3, 51, 20, 51, 42, 52, 12, 4, 7, 59, 58, 25, 94, 18]

In [57]:
def NFA(cargos: Iterable[int], container_size: int) -> Dict:
    containers = defaultdict(list)
    container_n = 1
    cur_weight = 0
    for c in cargos:
        if cur_weight + c > container_size:
            cur_weight = c
            container_n += 1
        else:
            cur_weight += c
        containers[container_n].append(c)
    return containers

In [58]:
NFA(cargos, 100)

defaultdict(list,
            {1: [23, 3, 51, 20],
             2: [51, 42],
             3: [52, 12, 4, 7],
             4: [59],
             5: [58, 25],
             6: [94],
             7: [18]})

In [85]:
def FFA(cargos: Iterable[int], container_size: int) -> defaultdict:
    containers = defaultdict(list)
    containers_weight = defaultdict(int)
    container_n = 1
    containers_weight[container_n] = 0
    
    # helper function to find and fill avail containers
    def find_free_cont(c):
        for k, v in containers_weight.items():
            if v + c <= container_size:
                containers[k].append(c)
                containers_weight[k] += c
                return True
        return False

    for c in cargos:
        if containers_weight[container_n] + c > container_size:
            if not find_free_cont(c):
                container_n += 1
                containers_weight[container_n] = c
                containers[container_n].append(c)
        else:
            containers_weight[container_n] += c
            containers[container_n].append(c)
        
    return containers

In [87]:
FFA(cargos, 100)

[23, 3, 51, 20, 51, 42, 52, 12, 4, 7, 59, 58, 25, 94, 18]


defaultdict(list,
            {1: [23, 3, 51, 20],
             2: [51, 42],
             3: [52, 12, 4, 7, 18],
             4: [59],
             5: [58, 25],
             6: [94]})

In [90]:
def WFA(cargos: Iterable[int], container_size: int) -> defaultdict:
    containers = defaultdict(list)
    containers_weight = defaultdict(int)
    container_n = 1
    containers_weight[container_n] = 0
    
    # helper function to get min weight container
    def find_min_weight(c):
        k = min(containers_weight.items(), key=itemgetter(1))[0]
        if containers_weight[k] + c <= container_size:
            containers[k].append(c)
            containers_weight[k] += c
            return True
        return False

    for c in cargos:
        if containers_weight[container_n] + c > container_size:
            if not find_min_weight(c):
                container_n += 1
                containers_weight[container_n] = c
                containers[container_n].append(c)
        else:
            containers_weight[container_n] += c
            containers[container_n].append(c)
        
    return containers

In [91]:
WFA(cargos, 100)

defaultdict(list,
            {1: [23, 3, 51, 20],
             2: [51, 42],
             3: [52, 12, 4, 7],
             4: [59, 18],
             5: [58, 25],
             6: [94]})

In [104]:
def generic_alg(cargos: Iterable[int], container_size: int) -> defaultdict:
    containers = defaultdict(list)
    containers_weight = defaultdict(int)
    container_n = 1
    containers_weight[container_n] = 0
    
    # helper function to get min weight container
    def find_max_avail_weight(c):
        # max_weight = 0
        max_weight_k = None
        for k, v in containers_weight.items():
            if v + c <= container_size:
                if max_weight_k is None or containers_weight[max_weight_k] < v:
                    max_weight_k = k
       #  print(max_weight_k)
        return max_weight_k


    for c in cargos:
        to_append = container_n
        if containers_weight[to_append] + c > container_size:
            to_append = find_max_avail_weight(c)
            if to_append is None:
                # can't append cargo to existing container, so add new
                container_n += 1
                containers_weight[container_n] = c
                containers[container_n].append(c)
                continue
        containers_weight[to_append] += c
        containers[to_append].append(c)
        
    return containers

In [105]:
BFA(cargos, 100)

defaultdict(list,
            {1: [23, 3, 51, 20],
             2: [51, 42],
             3: [52, 12, 4, 7, 18],
             4: [59],
             5: [58, 25],
             6: [94]})